有代码运行环境，那么我可以直接用以下流程：
增加一个字典，使用导入写


# 开源代码

### 第一步planner

In [30]:
from planner.tools import get_pla_user
from planner.prompt import get_planer
from utils.api import gpt_chat
type1="opt"
question="./test_case/o4/question.txt"

planer = get_planer(problem_type=type1)
info = get_pla_user(ques=question,problem_type=type1)

# response1 = gpt_chat(sys=planer,user=info,provider="deepseek")


In [16]:
def write_file(file_path: str, content):
    """
    Write content to a file based on its extension.
    
    Args:
        file_path: Path to the file to be written
        content: Content to write (string for txt/md, list of strings for csv)
    """
    if file_path.endswith('txt'):
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content if isinstance(content, str) else '\n'.join(content))
    elif file_path.endswith('csv'):
        with open(file_path, 'w', encoding='utf-8', newline='') as f:
            if isinstance(content, list):
                f.writelines(content)
            else:
                f.write(content)
    elif file_path.endswith('md'):
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content if isinstance(content, str) else '\n'.join(content))
    else:
        raise ValueError("Unsupported file format")

In [17]:
# 读取文件内容
def read_file(file_path: str):
    """
    Read the content of a file and return it as a string.
    """
    if file_path.endswith('txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    if file_path.endswith('csv'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.readlines()
    if file_path.endswith('md'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

## 匹配函数

In [31]:
from planner.tools import get_plan
from utils.rag import ChoromaDBManager


chroma_db = ChoromaDBManager("./test_case/o4/tool_db")
chroma_db.store_tools_to_db(dir_path="./tool_doc_md")

✅ 已存储: statistic.md (包含 2 个工具块)
✅ 已存储: data_clean.md (包含 5 个工具块)
✅ 已存储: graph_optimization.md (包含 5 个工具块)
✅ 已存储: machine_learning.md (包含 15 个工具块)
✅ 已存储: math_optimization.md (包含 10 个工具块)
✅ 已存储: feature_process.md (包含 7 个工具块)
✅ 已存储: evaluate_model.md (包含 3 个工具块)


In [32]:
plan = get_plan(str_path="./test_case/o4/plan_deepseek.txt")
funcs=chroma_db.get_all_tools(plan)

In [33]:
funcs

[{'tool_name': 'solve_lp',
  'content': '**Name:** solve_lp  \n**Description:** 用于简单线性规划问题，线性规划求解函数。\n**Applicable Situations:**  \n- 需要求解目标函数最小化的线性规划问题  \n\n**Parameters:**  \n- `c`:  \n  - \u200b**Type:** `array-like`  \n  - \u200b**Description:** 目标函数系数向量，维度需与变量数一致  \n\n- `A_ub`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 不等式约束系数矩阵（默认None），每行对应一个约束条件  \n\n- `b_ub`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 不等式约束右侧常数向量（默认None）  \n\n- `A_eq`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 等式约束系数矩阵（默认None）  \n\n- `b_eq`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 等式约束右侧常数向量（默认None）  \n\n- `bounds`:  \n  - \u200b**Type:** `list of tuples | None`  \n  - \u200b**Description:** 变量边界列表（默认None），每个元组表示变量下界和上界  \n  - \u200b**Example:** `[(0, None), (0, 5)]` 表示x₁≥0，x₂∈[0,5]  \n\n**Result:**  \n- 成功时返回包含最优解x和最优值fun的元组  \n- 失败时返回包含错误信息的字符串  \n\n**Example Call:**  \n```python\

## Developer

In [34]:
from developer.prompt import get_developer
from developer.tools import get_dev_user
funcs = [{'tool_name': 'solve_lp',
  'content': '**Name:** solve_lp  \n**Description:** 用于简单线性规划问题，线性规划求解函数。\n**Applicable Situations:**  \n- 需要求解目标函数最小化的线性规划问题  \n\n**Parameters:**  \n- `c`:  \n  - \u200b**Type:** `array-like`  \n  - \u200b**Description:** 目标函数系数向量，维度需与变量数一致  \n\n- `A_ub`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 不等式约束系数矩阵（默认None），每行对应一个约束条件  \n\n- `b_ub`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 不等式约束右侧常数向量（默认None）  \n\n- `A_eq`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 等式约束系数矩阵（默认None）  \n\n- `b_eq`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 等式约束右侧常数向量（默认None）  \n\n- `bounds`:  \n  - \u200b**Type:** `list of tuples | None`  \n  - \u200b**Description:** 变量边界列表（默认None），每个元组表示变量下界和上界  \n  - \u200b**Example:** `[(0, None), (0, 5)]` 表示x₁≥0，x₂∈[0,5]  \n\n**Result:**  \n- 成功时返回包含最优解x和最优值fun的元组  \n- 失败时返回包含错误信息的字符串  \n\n**Example Call:**  \n```python\n# 求解 min -x1 + 4x2 \n# s.t.  x1 + x2 <= 5\n#       2x1 + x2 = 4\n#       x1 ∈ [0,3], x2 ≥ 0\nx_opt, f_opt = solve_lp(\n    c=[-1, 4],\n    A_ub=[[1, 1]],\n    b_ub=[5],\n    A_eq=[[2, 1]],\n    b_eq=[4],\n    bounds=[(0, 3), (0, None)]\n)\n```',
  'source_file': 'math_optimization.md',
  'distance': 0.5363641381263733}]
devloper_prompt = get_developer(problem_type=type1,func=funcs)
user2 = get_dev_user(question=question,problem_type=type1)

# response2 = gpt_chat(sys=devloper_prompt,user=user2)

## 编译

In [35]:
from utils.notebook_serializer import NotebookSerializer
from utils.local_interpreter import LocalCodeInterpreter
import sys
import json
import os
dirname = "./"
notebook = NotebookSerializer(dirname)

# agents code执行
code_interpreter = LocalCodeInterpreter(work_dir=dirname,notebook_serializer=notebook,task_id="111")
code_interpreter.initialize()

In [36]:
import importlib

for func in funcs:
    file = set()
    if func["source_file"] not in file:
        module_name = f"tool_code.{func['source_file'].replace('.md', '')}"
        module = importlib.import_module(module_name)
        header = module.get_header()
        tool = f"from tool_code.{func['source_file'].replace('.md', '')} import {func['tool_name']}"
        code_interpreter.execute_code(header)
        code_interpreter.execute_code(tool)
        file.add(func['source_file'])

In [37]:
from utils.utils1 import extract_functions

functions = extract_functions("./test_case/o4/dev_deepseek.txt")

In [6]:
import ast
def convert_str_function(json_str):
    data = ast.literal_eval(json_str)
    results = []
    for item in data:
        name = item['name']
        args = item['args']
        arg_strs = []
        for k, v in args.items():
            # 保证字典等非字符串不被误转义为字符串
            if isinstance(v, str):
                arg_strs.append(f'{k}="{v}"')
            else:
                arg_strs.append(f"{k}={v}")
        arg_str = ", ".join(arg_strs)
        call_str = f'result = {name}({arg_str})'
        results.append(call_str)

    return "\n".join(results)

### 两步

In [8]:
import re
func_call = convert_str_function(functions[0])
data_processed = func_call.split("\n")
updated_lines = []

train = 0
test = 0

for tmp in data_processed:
    if 'data="train_data"' in tmp:
        if train == 0:
            data_name = 'train_data'
        else:
            data_name = f'train_data{train}'
        output_var = f'train_data{train + 1}'
        tmp = tmp.replace('data="train_data"', f'data={data_name}')
        tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
        train += 1
    elif 'data="test_data"' in tmp:
        if test == 0:
            data_name = 'test_data'
        else:
            data_name = f'test_data{test}'
        output_var = f'test_data{test + 1}'
        tmp = tmp.replace('data="test_data"', f'data={data_name}')
        tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
        test += 1
    updated_lines.append(tmp)
    

In [9]:
updated_lines

['train_data1 = fill_missing_values(data=train_data, columns=[\'HomePlanet\', \'CryoSleep\', \'Destination\', \'Age\', \'VIP\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], strategy="mode")',
 'test_data1 = fill_missing_values(data=test_data, columns=[\'HomePlanet\', \'CryoSleep\', \'Destination\', \'Age\', \'VIP\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], strategy="mode")',
 'train_data2 = detect_and_handle_outliers_zscore(data=train_data1, columns=[\'Age\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], threshold=3.0, method="clip")',
 'test_data2 = detect_and_handle_outliers_zscore(data=test_data1, columns=[\'Age\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], threshold=3.0, method="clip")',
 "train_data3 = one_hot_encode(data=train_data2, columns=['HomePlanet', 'CryoSleep', 'Destination', 'VIP'])",
 "test_data3 = one_hot_encode(data=test_data2, columns=['HomePlanet', 'CryoS

In [10]:
data_load ="""train_data = pd.read_csv("{train_path}")
test_data = pd.read_csv("{test_path}")"""
path1 = "./test_case/p1/train.csv"
path2 = "./test_case/p1/test.csv"
data_load=data_load.format(train_path=path1,test_path=path2)

code_interpreter.execute_code(data_load)

('', False, '')

In [11]:
for i in updated_lines:
    code_interpreter.execute_code(i)

In [14]:
model = convert_str_function(functions[1])
model1=model.replace('"train_data"', f'train_data{train}').replace('"test_data"', f'test_data{test}')

In [15]:
code_interpreter.execute_code(model1)

('---------------------------------------------------------------------------\nKeyError                                  Traceback (most recent call last)\nFile ~/anaconda3/envs/base1/lib/python3.10/site-packages/xgboost/data.py:407, in pandas_feature_info(data, meta, feature_names, feature_types, enable_categorical)\n    406 try:\n--> 407     new_feature_types.append(_pandas_dtype_mapper[dtype.name])\n    408 except KeyError:\n\nKeyError: \'object\'\n\nDuring handling of the above exception, another excepti\n... (内容已截断) ...\n3.10/site-packages/xgboost/data.py:372, in _invalid_dataframe_dtype(data)\n    370 type_err = "DataFrame.dtypes for data must be int, float, bool or category."\n    371 msg = f"""{type_err} {_ENABLE_CAT_ERR} {err}"""\n--> 372 raise ValueError(msg)\n\nValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:PassengerId

### 单步

In [39]:
functions[0]

'[    {        "name": "solve_lp",        "args": {            "c": [-500, -550, -630, -1000, -800, -700, -800, -700, -600, -950, -900, -930, -1000, -960, -840, -650, -600, -700, -1200, -1040, -980, -860, -880, -780],            "A_ub": [                [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],                [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],                [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],                [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],                [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],                [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 

In [38]:
model = convert_str_function(functions[0])
model

ValueError: malformed node or string on line 1: <ast.BinOp object at 0x7f74b7851360>

In [26]:
code_interpreter.execute_code(model)

('', False, '')

## 汇总

In [29]:
from planner.tools import get_pla_user
from planner.prompt import get_planer
from utils.api import gpt_chat
from utils.utils1 import read_file,write_file,extract_functions,convert_str_function
import os
from planner.tools import get_plan
from utils.rag import ChoromaDBManager
from developer.prompt import get_developer
from developer.tools import get_dev_user
from utils.notebook_serializer import NotebookSerializer
from utils.local_interpreter import LocalCodeInterpreter
import sys
import json
import importlib
import re

def func_preprocess(functions):
    func_call = convert_str_function(functions[0])
    data_processed = func_call.split("\n")
    updated_lines = []

    train = 0
    test = 0

    for tmp in data_processed:
        if 'data="train_data"' in tmp:
            if train == 0:
                data_name = 'train_data'
            else:
                data_name = f'train_data{train}'
            output_var = f'train_data{train + 1}'
            tmp = tmp.replace('data="train_data"', f'data={data_name}')
            tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
            train += 1
        elif 'data="test_data"' in tmp:
            if test == 0:
                data_name = 'test_data'
            else:
                data_name = f'test_data{test}'
            output_var = f'test_data{test + 1}'
            tmp = tmp.replace('data="test_data"', f'data={data_name}')
            tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
            test += 1
        updated_lines.append(tmp)
    return updated_lines,train,test

def code_exe(prepare,type,functions,question):
    notebook = NotebookSerializer("./")
    code_interpreter = LocalCodeInterpreter(work_dir="./",notebook_serializer=notebook,task_id="111")
    code_interpreter.initialize()

    for func in prepare:
        file = set()
        if func["source_file"] not in file:
            module_name = f"tool_code.{func['source_file'].replace('.md', '')}"
            module = importlib.import_module(module_name)
            header = module.get_header()
            tool = f"from tool_code.{func['source_file'].replace('.md', '')} import {func['tool_name']}"
            code_interpreter.execute_code(header)
            code_interpreter.execute_code(tool)
            file.add(func['source_file'])
    if (len(functions) > 1) and (type in ["pre","eval"]):
        update_func,train_index,test_index = func_preprocess(functions)
        data_load ="""train_data = pd.read_csv("{train_path}")
        test_data = pd.read_csv("{test_path}")"""
        path1 = os.path.join(os.path.dirname(question),"train.csv")
        path2 = os.path.join(os.path.dirname(question),"test.csv")
        data_load=data_load.format(train_path=path1,test_path=path2)
        code_interpreter.execute_code(data_load)
        for i in update_func:
            code_interpreter.execute_code(i)
        model = convert_str_function(functions[1])
        model1=model.replace('"train_data"', f'train_data{train_index}').replace('"test_data"', f'test_data{test_index}')
        code_interpreter.execute_code(model1)
    else:
        model = convert_str_function(functions[0])
        code_interpreter.execute_code(model)
        code_interpreter.execute_code("print(result)")



if __name__ == "__main__":
    type1="opt"
    question="./test_case/o4/question.txt"
    agent ="deepseek"
    # planner环节
    print("planner start")
    planer = get_planer(problem_type=type1)
    info = get_pla_user(ques=question,problem_type=type1)
    # response1 = gpt_chat(sys=planer,user=info,provider="deepseek")
    # write_file(os.path.join(os.path.dirname(question),f"plan_{agent}.txt"), response1)
    response1 = read_file(os.path.join(os.path.dirname(question),f"plan_{agent}.txt"))

    # RAG环节
    print("RAG start")
    chroma_db = ChoromaDBManager(os.path.join(os.path.dirname(question), "tool_db"))
    chroma_db.store_tools_to_db(dir_path="./tool_doc_md")
    plan = get_plan(str_path=os.path.join(os.path.dirname(question),f"plan_{agent}.txt"))
    prepare_funcs=chroma_db.get_all_tools(plan)

    #developer环节
    print("developer start")
    devloper_prompt = get_developer(problem_type=type1,func=prepare_funcs)
    user2 = get_dev_user(question=question,problem_type=type1)
    # response2 = gpt_chat(sys=devloper_prompt,user=user2)
    # write_file(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"), response2)
    response2 = read_file(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"))

    # 代码执行环节
    print("code execute start")
    exec_func = extract_functions(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"))
    code_exe(prepare=prepare_funcs,type=type1,functions=exec_func,question=question)


planner start
RAG start
✅ 已存储: statistic.md (包含 2 个工具块)
✅ 已存储: data_clean.md (包含 5 个工具块)
✅ 已存储: graph_optimization.md (包含 5 个工具块)
✅ 已存储: machine_learning.md (包含 15 个工具块)
✅ 已存储: math_optimization.md (包含 10 个工具块)
✅ 已存储: feature_process.md (包含 7 个工具块)
✅ 已存储: evaluate_model.md (包含 3 个工具块)
developer start
code execute start


ValueError: malformed node or string on line 1: <ast.BinOp object at 0x7f76d8167880>